# Описание задачи

Предложенный алгоритм берет за основу задачу maxcut. После сжатия и конволюции пикселям картинки сопоставляются бинарные переменные, отвечающие за принадлежность к черному или белому множеству. Далее ставится задача минимизации количества рёбер между двумя множествами при максимизации весов этих рёбер. Рёбра из каждого узла ведём к его соседям (соседи определяются отдельной функцией). Вес же ребра (i, j) выбираем пропорциональным $(x_i - x_j)^2 - w\,(\mathrm{imgData}[i] - \mathrm{imgData}[j])^2 (x_i - x_j)^2$.

Оценка масштабируемости алгоритма зависит от конкретного класса картинок, а точнее от необходимой степени детализации. Количество переменных в CUBO определяется числом пикселей картинки NxM. В случае отсутствия необходимости в высокой детализации любая картинка может быть значительно сжата, что положительно отражается на производительности. Тем не менее обычно число пикселей задаёт множество переменных превосходящее таковые в других задачах, поэтому проблему сегментации, вероятно, стоит относить к ресурсозатратным. Сходится выбранный алгоритм достаточно быстро: не было обнаружено улучшения результата при увеличении параметра num_reads выше 40.

В качестве классического аналога мы предлагаем сегментацию машинным обучением. Визуально качество работы класического алгоритма выглядит лучше чем квантового

# Классическое машинное обучение

In [701]:
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
import random
import pickle
import neal
from scipy import sparse as sp
from scipy import signal as sg

from PIL import Image as image

def loadData(filePath, conv=1, comp=1, show=False):
    f = open(filePath,'rb') 
    data = []
    
    img = image.open(f) 
    m,n = img.size
    
    m /= comp
    n /= comp
    
    m = int(m)
    n = int(n)
    
    img = img.resize((m, n))
    if(show):
        img.show()
    
    tmp = np.zeros((m, n))
    
    for i in range(m):
        for j in range(n):
            tmp[i, j] = img.getpixel((i,j))
    f.close()
    
    data = sg.convolve2d(tmp, np.ones((conv, conv)), mode='valid')/conv**2
    
    row = data.shape[0]
    col = data.shape[1]
    
    pic_new = image.new("L",(row,col))
    
    for i in range(row):
        for j in range(col):
            pic_new.putpixel((i,j), int(data[i, j]))
    if(show):
        pic_new.show()
    
    return data,row,col

In [657]:
from sklearn.cluster import  KMeans
km=KMeans(n_clusters=2)

imgData,row,col =loadData('heraldic_inv.png')
imgData = imgData.reshape(row*col, 1)

label = km.fit_predict(imgData)
label=label.reshape([row,col])

pic_new = image.new("L",(row,col))
for i in range(row):
    for j in range(col):
        pic_new.putpixel((i,j),int(256*label[i][j]))
pic_new.save("name.jpg","JPEG")
pic_new.show()

In [640]:
%ls

1.png                  4.png                  grapes_inv.png
1_small.png            KM_Apple.jpg           heraldic_inv.png
1_small_segmented.png  Q3.pkl                 normal_name.jpg
2.png                  Q3sp.pkl               solution.yaml
2_small_c.png          Task_3.ipynb           targets/
3.png                  bloch_inv.png


# Квантовый алгоритм CUBO

In [658]:
def d_circle(i, row, d):
    
    # возвращает индексы узлов в квадрате сполуребром = d
    
    ans = []
    
    # верхнее ребро
    ans += [k + i - d*row for k in list(range(-d, d+1))]
    
    # правое ребро
    ans += [k*row + i + d for k in list(range(-d, d+1))]
    
    # нижнее ребро
    ans += [-k + i + d*row for k in list(range(-d, d+1))]
    
    # левое ребро
    ans += [-k*row + i - d for k in list(range(-d, d+1))]
    
    np.asarray(ans)
    
    return ans

In [706]:
imgData,row,col =loadData('1.png', conv=2, comp=1.5)

imgData = imgData.reshape(row*col)
imgData = imgData/imgData[imgData.argmax()]

N = imgData.shape[0]

In [707]:
w_1 = 6
w_2 = 20
w_3 = 25

from scipy import sparse as sp

K = np.zeros((N, N))

# заполнение квадратичной формы
for i in range(N):
    
    #print(i, '\n')
    neighbors = d_circle(i, row, d=2)
    
    for n in neighbors:
        
        if(n >= 0 and n < N):

            tmp = 1 - w_2*(imgData[i] - imgData[n])**2

            K[i, i] += tmp
            K[i, n] -= tmp

    neighbors = d_circle(i, row, d=1)
    
    for n in neighbors:
        
        if(n >= 0 and n < N):

            tmp = 1 - w_1*(imgData[i] - imgData[n])**2

            K[i, i] += tmp
            K[i, n] -= tmp
            
    neighbors = d_circle(i, row, d=3)
    
    for n in neighbors:
        
        if(n >= 0 and n < N):

            tmp = 1 - w_3*(imgData[i] - imgData[n])**2

            K[i, i] += tmp
            K[i, n] -= tmp

In [709]:
sampler = neal.SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(Q=K, num_reads=100)
dict_out = list(sampleset.lowest().data())[0][0]
val_old = np.asarray(list(dict_out.values()))

val = val_old.reshape(row, col)
pic_new = image.new("L",(row,col))

for i in range(row):
    for j in range(col):
        pic_new.putpixel((i,j),int(256*(1-val[i][j])))
                         
pic_new.save("1_new.jpg","JPEG")
pic_new.show()

In [ ]:
# конвертер в файл
K = sp.csr_matrix(K)

# Оптимизация весов (обучение)

In [720]:
from scipy.optimize import minimize

def single_loss(x, pic, target):
    
    conv = int(x[0])
    w_1 = x[1]
    w_2 = x[2]
    w_3 = x[3]
    
    imgData,row,col =loadData(pic, conv=conv, comp=1.5)

    imgData = imgData.reshape(row*col)
    imgData = imgData/imgData[imgData.argmax()]

    N = imgData.shape[0]
    
    K = np.zeros((N, N))

    # заполнение квадратичной формы
    for i in range(N):

        #print(i, '\n')
        neighbors = d_circle(i, row, d=2)

        for n in neighbors:

            if(n >= 0 and n < N):

                tmp = 1 - w_2*(imgData[i] - imgData[n])**2

                K[i, i] += tmp
                K[i, n] -= tmp

        neighbors = d_circle(i, row, d=1)

        for n in neighbors:

            if(n >= 0 and n < N):

                tmp = 1 - w_1*(imgData[i] - imgData[n])**2

                K[i, i] += tmp
                K[i, n] -= tmp

        neighbors = d_circle(i, row, d=3)

        for n in neighbors:

            if(n >= 0 and n < N):

                tmp = 1 - w_3*(imgData[i] - imgData[n])**2

                K[i, i] += tmp
                K[i, n] -= tmp
                
    sampler = neal.SimulatedAnnealingSampler()
    sampleset = sampler.sample_qubo(Q=K, num_reads=40)
    dict_out = list(sampleset.lowest().data())[0][0]
    val_old = np.asarray(list(dict_out.values()))

    val = val_old.reshape(row, col)
    pic_new = image.new("L",(row,col))

    for i in range(row):
        for j in range(col):
            pic_new.putpixel((i,j),int(256*(1-val[i][j])))

    f = open(target,'rb') 
    imgData = image.open(f) 
    imgData = imgData.resize((row, col))
    
    ans=0
    
    for n in range(row):
        for m in range(col):
            
            ans += (pic_new.getpixel((n, m)) - imgData.getpixel((n, m)))**2
            
    return ans

def loss(x):
    
    ans = 0
    ans += single_loss(x, '1_small.png', 'targets/1_small_target.png')
    ans += single_loss(x, '2_small_c.png', 'targets/2_small_c_target.png')
    ans += single_loss(x, '2.png', 'targets/2_target.png')
    ans += single_loss(x, 'bloch_inv.png', 'targets/bloch_inv_target.png')
    ans += single_loss(x, 'heraldic_inv.png', 'targets/heraldic_inv_target.png')
    
    return ans

abs_loss = 1e10

for i in range(10):
    
    borders = np.asarray([[1, 9], [0, 250], [0, 250], [0, 250]])
    # генерируем x0
    x0 = np.random.rand(4) * (borders[:, 1] - borders[:, 0]) + borders[:, 0]
    # оптимизируем
    sol = minimize(loss, x0=x0, bounds=borders)

    if(sol.fun < abs_loss):
        np.savez('opt_params', loss=sol.fun, x=sol.x)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/andreykugut/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/d0/tt3hk6g15m78sc462y4r9kk80000gn/T/ipykernel_29716/2288319933.py", line 100, in <cell line: 94>
    sol = minimize(loss, x0=x0, bounds=borders)
  File "/Users/andreykugut/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_minimize.py", line 623, in minimize
    return _minimize_lbfgsb(fun, x0, args, jac, bounds,
  File "/Users/andreykugut/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/lbfgsb.py", line 306, in _minimize_lbfgsb
    sf = _prepare_scalar_function(fun, x0, jac=jac, args=args, epsilon=eps,
  File "/Users/andreykugut/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/optimize.py", line 261, in _prepare_scalar_function
    sf = ScalarFunction(fun, x0, args, grad, hess,
  File "/Users/andreykugut/opt/anaconda3/lib/python3

In [676]:
%ls

1.png                  4.png                  grapes_inv.png
1_small.png            KM_Apple.jpg           heraldic_inv.png
1_small_segmented.png  Q3.pkl                 normal_name.jpg
2.png                  Q3sp.pkl               solution.yaml
2_small_c.png          Task_3.ipynb           targets/
3.png                  bloch_inv.png
